## Configuring and Running BG Compliance
In this example we demonstrate the configuration of several pre-trade rules. 
We will create the rules for a portfolio group containing a lookthrough portfolio and a normal portfolio. 

The rules that we are implementing are the below:
1) Investment in Chinese stock index futures is not permitted
2) Fixed income funds should not buy securitisation bonds issued after 1st January 2019
3) The minimum permitted holding in bonds issued by emerging market issuers or in emerging market currencies is 80%
4) the minimum permitted holding in bonds lower than BBB- is 10%

We will set up these rules using the datapoints available to us and run them against BG comp portfolio group. Then we will raise some orders on that portfolio and re-check pre-trade compliance. 

In [18]:
import lusid
import lusid.api as la
import lusid.models as lm
from lusid.models.upsert_compliance_rule_request import UpsertComplianceRuleRequest
from lusid.models.reference_list_request import ReferenceListRequest
from lusid import ApiException
from lusid.utilities import ApiClientFactory
from lusidjam.refreshing_token import RefreshingToken
from lusidtools.cocoon.cocoon import load_from_data_frame
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.seed_sample_data import seed_data
from lusidtools.cocoon.utilities import create_scope_id
from lusidtools.cocoon.cocoon_printer import (
    format_instruments_response,
    format_portfolios_response,
    format_transactions_response,
    format_quotes_response,
    format_holdings_response,
)
from collections import defaultdict
import pandas as pd
import numpy as np
import json
import openpyxl
import os
import datetime
from datetime import datetime, timedelta, time, date
import pytz

pd.set_option('display.max_columns', None)

# Authenticate our user and create our API client
secrets_path = os.getenv("FBN_SECRETS_PATH")

# Initiate an API Factory which is the client side object for interacting with LUSID APIs
api_factory = lusid.utilities.ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path,
    app_name="LusidJupyterNotebook")

print ('LUSID Environment Initialised')
print ('API Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
API Version:  0.6.12613.0


In [19]:
# define some APIs
properties_api = api_factory.build(la.PropertyDefinitionsApi)
referencelist_api = api_factory.build(la.ReferenceListsApi)
compliance_api = api_factory.build(la.ComplianceApi)
aggregation_api = api_factory.build(la.AggregationApi)
configuration_recipe_api = api_factory.build(la.ConfigurationRecipeApi)
portfolio_groups_api = api_factory.build(la.PortfolioGroupsApi)
portfolios_api = api_factory.build(la.PortfoliosApi)
instruments_api = api_factory.build(la.InstrumentsApi)
allocations_api = api_factory.build(lusid.api.AllocationsApi)
transaction_portfolios_api = api_factory.build(la.TransactionPortfoliosApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)


### 1 Create portfolio and portfolio groups
Currently the compliance rules runs on portgroup. Since we are running the rule on BG EM bond only. we will create a portgroup with only this fund in it. 

In [20]:
#define scope and portgroup
scope = 'BG_PMS_DEMO'
portfolio_code = "89518"
portfolio_group_code='Baillie_Comp'
portfolio_code2="89520"
portfolio_name="Baillie Gifford fund Lookthrough" #lookthrough portfolio
portfolio_base_currency='GBP'
child_portfolio = '148993'

In [21]:
def create_portfolio(scope, portfolio_code, name):

    pf_df = pd.DataFrame(data=[
        {"portfolio_code": portfolio_code2, "portfolio_name": portfolio_name},
    ])
    
    portfolio_mapping = {
        "required": {
            "code": "portfolio_code",
            "display_name": "portfolio_name",
            "base_currency": '$GBP',
        },
        "optional": {
            "created": "$2020-01-01T00:00:00+00:00"
        },
    }
    
    result = load_from_data_frame(
        api_factory=api_factory,
        scope=scope,
        data_frame=pf_df,
        mapping_required=portfolio_mapping["required"],
        mapping_optional=portfolio_mapping["optional"],
        file_type="portfolios",
    )

    succ, failed = format_portfolios_response(result)
    display(pd.DataFrame(data=[{"success": len(succ), "failed": len(failed)}])) 

In [22]:
def create_portfolio_group(portfolio_groups_api, scope, code, portfolios):

    portfolio_creation_date = datetime.now(pytz.UTC) - timedelta(days=5000)

    try:
        portfolio_groups_api.delete_portfolio_group(
            scope=scope,
            code=code)
    except:
        pass
    
    group_request = lm.CreatePortfolioGroupRequest(
        code=code,
        display_name=code,
        values=portfolios,
        sub_groups=None,
        description=None,
        created=portfolio_creation_date)

    portfolio_group = portfolio_groups_api.create_portfolio_group(
        scope=scope,
        create_portfolio_group_request=group_request)
    
    return portfolio_group

In [23]:
create_portfolio(scope, portfolio_code, portfolio_name)

,success,failed
0,1,0


In [36]:
result = create_portfolio_group(portfolio_groups_api, scope, portfolio_group_code, [lm.ResourceId(scope=scope, code=portfolio_code2),lm.ResourceId(scope=scope, code=portfolio_code)])

Create lookthrough portfolio that has SI Equity fund as the underlying portfolio by securitising the portfolio.

In [25]:
upsert_instrument = instruments_api.upsert_instruments(
    request_body={
        "child instrument": lm.InstrumentDefinition(
            name="child instrument",
            identifiers={"ClientInternal": lm.InstrumentIdValue(value="ChildInstrument")},
            look_through_portfolio_id={'scope':scope,'code':child_portfolio},
            definition={'instrumentType':'SimpleInstrument',
                        'domCcy':'GBP',
                        'assetClass':'Unknown',
                        'simpleInstrumentType':'Fund'}),
    }
)
upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instrument.values.values()))
display(upsert_instruments_response_df[["name", "lusid_instrument_id"]])

,name,lusid_instrument_id
0,child instrument,LUID_00003HI5


Insert transaction to the lookthrough portfolio

In [33]:
# Set trade variables
trade_date = datetime(2024, 2, 16, tzinfo=pytz.utc)
settle_days = 2
units = 10

# Book a StockIn transaction against the forward
fwd_txn = lm.TransactionRequest(
    transaction_id="TXN009",
    type="StockIn",
    instrument_identifiers={"Instrument/default/ClientInternal": 'ChildInstrument'},
    transaction_date=trade_date.isoformat(),
    settlement_date=(trade_date + timedelta(days=settle_days)).isoformat(),
    units=units,
    transaction_price=lm.TransactionPrice(price=200,type="Price"),
    total_consideration=lm.CurrencyAndAmount(amount=10000,currency="GBP"),
    exchange_rate=1,
    transaction_currency="GBP"
)

response = transaction_portfolios_api.upsert_transactions(scope=scope,
                                                    code=portfolio_code2,
                                                    transaction_request=[fwd_txn])

print(f"Transaction successfully updated at time: {response.version.as_at_date}")

Transaction successfully updated at time: 2024-02-19 11:01:30.049218+00:00


In [66]:
#only use if you need to adjust holdings
transaction_portfolios_api.set_holdings(scope=scope,
                                        code=portfolio_code2,
                                        effective_at="2024-02-16",
                                        adjust_holding_request=[
            lm.AdjustHoldingRequest(
                instrument_identifiers={"Instrument/default/ClientInternal": 'ChildInstrument2'},
                sub_holding_keys=None,
                properties=None,
                tax_lots=[lm.TargetTaxLotRequest(units=0)],
                currency="GBP",
            )
        ]
    )

{'href': 'https://bg-pms-testing-demo.lusid.com/api/api/transactionportfolios/BG_PMS_DEMO/89520/holdings?asAt=2024-02-16T10%3A13%3A50.4944240%2B00%3A00',
 'links': [{'description': None,
            'href': 'https://bg-pms-testing-demo.lusid.com/api/api/portfolios/BG_PMS_DEMO/89520?effectiveAt=2020-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2024-02-16T10%3A13%3A50.4944240%2B00%3A00',
            'method': 'GET',
            'relation': 'Root'},
           {'description': None,
            'href': 'https://bg-pms-testing-demo.lusid.com/api/api/transactionportfolios/BG_PMS_DEMO/89520/holdings?effectiveAt=2020-01-01T00%3A00%3A00.0000000%2B00%3A00&asAt=2024-02-16T10%3A13%3A50.4944240%2B00%3A00',
            'method': 'GET',
            'relation': 'Holdings'},
           {'description': 'A link to the LUSID Insights website showing all '
                           'logs related to this request',
            'href': 'https://bg-pms-testing-demo.lusid.com/app/insights/logs/0HN1ETP1EF7QC:00000

Lookthrough-enable recipe

In [56]:
# Create look-through-enabled recipe
lookthrough_config_recipe = lm.ConfigurationRecipe(
    scope=scope,
    code="lookthrough",
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                quote_interval= "12M.0D",
                field="mid",
            ),
            lm.MarketDataKeyRule(
                key="FX.CurrencyPair.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Rate",
                quote_interval= "12M.0D",
                field="mid",
            ),
            lm.MarketDataKeyRule(
                key="FX.CurrencyPair.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Rate",
                quote_interval= "12M.0D",
                price_source= "BG",
                field="mid",
            ),
            lm.MarketDataKeyRule(
                key="Quote.ClientInternal.*",
                supplier="Lusid",
                data_scope=scope,
                quote_type="Price",
                quote_interval= "12M.0D",
                price_source= "BG",
                field="mid",
            ),
        ],
        options=lm.MarketOptions(
            default_supplier="Lusid",
            default_instrument_code_type="LusidInstrumentId",
            default_scope=scope,
            attempt_to_infer_missing_fx=True,
        ),
    ),
    pricing=lm.PricingContext(
        options = lm.PricingOptions(
            model_selection = lm.ModelSelection(
            library = 'Lusid',
            model = 'SimpleStatic'
            ),
            allow_partially_successful_evaluation = True),
    
        # toggle look through
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="Bond",
                model_options=lm.OpaqueModelOptions(data={}, model_options_type="OpaqueModelOptions"),
            ),
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="SimpleStatic",
                instrument_type="SimpleInstrument",
                model_options=lm.IndexModelOptions(portfolio_scaling="Sum", model_options_type="IndexModelOptions"),

            )
        ],
    ),
)

upsert_configuration_recipe_response = (
    configuration_recipe_api.upsert_configuration_recipe(
        upsert_recipe_request=lm.UpsertRecipeRequest(
            configuration_recipe=lookthrough_config_recipe
        )
    )
)

In [ ]:
#insert the quotes for SI Equity Income holdings
quotes_df = pd.read_csv("data/quote_file.csv")
# Compliance runs at latest asat
quotes_df['quote_date'] = quotes_df['quote_date'].apply(lambda s: datetime.now(pytz.UTC).strftime("%d-%b-%y"))
quotes_df.head()

Generate valuatiuon to check if there are any errors

In [57]:
def generate_valuation_request(valuation_effectiveAt, price_field):

    # Create the valuation request
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(
            scope=scope, code="lookthrough"
        ),
        metrics=[
            lm.AggregateSpec("Instrument/default/Name", "Value"),
            lm.AggregateSpec("Valuation/PvInReportCcy", "Proportion"),
            lm.AggregateSpec("Valuation/PvInReportCcy", "Sum"),
            lm.AggregateSpec("Holding/default/Units", "Sum"),
            lm.AggregateSpec("Aggregation/Errors", "Value"),
        ],
        group_by=["Instrument/default/Name"],
        filters=[lm.PropertyFilter('Instrument/default/Name', 'NotEquals', 'USD'),
                lm.PropertyFilter('Instrument/default/Name', 'NotEquals', 'USD Cash')
        ],
        portfolio_entity_ids=[
            lm.PortfolioEntityId(scope=scope, code=portfolio_code2)
        ],
        valuation_schedule=lm.ValuationSchedule(
            effective_at=valuation_effectiveAt.isoformat()
        ),
    )

    return valuation_request

In [58]:
#we will use the mid price
aggregation = aggregation_api.get_valuation(
    valuation_request=generate_valuation_request(
        datetime.now(pytz.UTC), "mid"
    )
)
pd.DataFrame(aggregation.data)


,Instrument/default/Name,Proportion(Valuation/PvInReportCcy),Sum(Valuation/PvInReportCcy),Sum(Holding/default/Units),Aggregation/Errors
0,Sonic Healthcare,0.027959,559.174611,30.024391,[]
1,Admiral Group,0.016857,337.141753,16.193168,[]
2,Cognex Corp,0.012052,241.031979,5.470118,[]
3,Starbucks Corp,0.013506,270.123949,3.466809,[]
4,TSMC,0.033755,675.090724,46.407272,[]
5,Apple,0.033623,672.464481,4.407577,[]
6,Albemarle Corp,0.019124,382.475024,2.179657,[]
7,Partners Group,0.020960,419.201724,0.566540,[]
8,T$ Uncommitted Cash,-0.005667,-113.339463,-4487.738361,[]
9,Greencoat UK Wind,0.013069,261.375136,181.133150,[]


### 2 Create instrument properties
Create properties for compliance rules.
Since some of the rule require a combination of several different fields, we will also create some derived properties. https://support.lusid.com/knowledgebase/article/KA-02192/en-us

In [32]:
def create_property_definition(properties_api, domain, scope, code, data_type):
    properties_api.create_property_definition(
        create_property_definition_request=lm.CreatePropertyDefinitionRequest(
            domain=domain,
            scope=scope,
            code=code,
            display_name=code,
            life_time="Perpetual",
            value_required=False,
            data_type_id=lm.resource_id.ResourceId(scope="system", code=data_type)
        )
    )

In [33]:
#create all the necesary instruments
try:
    create_property_definition(properties_api, "Instrument", scope, 'SecDesc', "string")
except:
    pass
try:
    create_property_definition(properties_api, "Instrument", scope, 'EMDM', "string")
except:
    pass
try:
    create_property_definition(properties_api, "Instrument", scope, 'Ratings', "string")
except:
    pass
try:
    create_property_definition(properties_api, "Instrument", scope, 'Securitized', "string")
except:
    pass
try:
    create_property_definition(properties_api, "Instrument", scope, 'MeetsDueDilligencePerEuRegs', "string")
except:
    pass

try:
    create_property_definition(properties_api, "Instrument", scope, 'Placeholder', "string")
except:
    pass

In [34]:
#create 3 derived properties
try:
    properties_api.create_derived_property_definition(
        create_derived_property_definition_request=lusid.models.CreateDerivedPropertyDefinitionRequest(
            domain="Instrument",
            scope=scope,
            code="CNFtIndex",
            display_name="FutIndex from China",
            data_type_id=lusid.ResourceId(scope="system", code="string"),
            derivation_formula=f"if(Properties[Instrument/BG_PMS_DEMO/SecDesc] eq 'FutIndex' and DomCcy eq 'CNY') then 'Yes' else 'No'",
        )
    )
except lusid.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])
    


Error creating Property Definition 'Instrument/BG_PMS_DEMO/CNFtIndex' because it already exists.


In [35]:
#create 3 derived properties
try:
    properties_api.create_derived_property_definition(
        create_derived_property_definition_request=lusid.models.CreateDerivedPropertyDefinitionRequest(
            domain="Instrument",
            scope=scope,
            code="ApprovedSecuritisation",
            display_name="Securitised Bond not approved by EU",
            data_type_id=lusid.ResourceId(scope="system", code="string"),
            derivation_formula=f"if(Properties[Instrument/BG_PMS_DEMO/Securitized] eq 'Yes' and Properties[Instrument/BG_PMS_DEMO/MeetsDueDilligencePerEuRegs] neq 'Yes') then 'Yes' else 'No'",
        )
    )
except lusid.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

Error creating Property Definition 'Instrument/BG_PMS_DEMO/ApprovedSecuritisation' because it already exists.


In [36]:
#create 3 derived properties
try:
    properties_api.create_derived_property_definition(
        create_derived_property_definition_request=lusid.models.CreateDerivedPropertyDefinitionRequest(
            domain="Instrument",
            scope=scope,
            code="HYbond",
            display_name="HY bond",
            data_type_id=lusid.ResourceId(scope="system", code="string"),
            derivation_formula=f"if(Properties[Instrument/BG_PMS_DEMO/Ratings] in 'BB','B','CCC','CC','C','D') then 'Yes' else 'No'",
        )
    )
except lusid.exceptions.ApiException as e:
    print(json.loads(e.body)["title"])

Error creating Property Definition 'Instrument/BG_PMS_DEMO/HYbond' because it already exists.


we will now add some properties to the holdings in the portfolio. Note that some of the instruments are in 'BG_PMS_DEMO' while others are in 'default' scope. We will run script to classify instruments to their respective scope and assign the property values

In [59]:

instruments=portfolio_groups_api.get_holdings_for_portfolio_group(scope='BG_PMS_DEMO', code=portfolio_group_code)
instruments_df = lusid_response_to_data_frame(instruments)

for i in instruments_df.index:
    if instruments_df.loc[i, 'currency'] in ['USD', 'EUR', 'GBP']:
        instruments_df.loc[i, 'emdm'] = 'DM'
    else:
        instruments_df.loc[i, 'emdm'] = 'EM'

for i in instruments_df.index:
    if instruments_df.loc[i, 'currency'] in ['USD', 'EUR', 'GBP']:
        instruments_df.loc[i, 'rating'] = 'AA'
    elif instruments_df.loc[i, 'currency'] in ['ZAR','THB','TRY','PHP']:
        instruments_df.loc[i, 'rating'] = 'B'
    elif instruments_df.loc[i, 'currency'] in ['INR','UAH','MXN']:
        instruments_df.loc[i, 'rating'] = 'CCC'
    else:
        instruments_df.loc[i, 'rating'] = 'BBB'

for i in instruments_df.index:
    if instruments_df.loc[i, 'instrument_uid'] == 'LUID_00003HI0':
        instruments_df.loc[i, 'class'] = 'FutIndex'
    else:
        instruments_df.loc[i, 'class'] = 'NA'
        
for i in instruments_df.index:
    if instruments_df.loc[i, 'instrument_uid'] in ['LUID_00003DMZ','LUID_00003DSA']:
        instruments_df.loc[i, 'securit'] = 'Yes'
    else:
        instruments_df.loc[i, 'securit'] = 'No'
        
for i in instruments_df.index:
    if instruments_df.loc[i, 'instrument_uid'] =='LUID_00003DMZ':
        instruments_df.loc[i, 'EU'] = 'Yes'
    else:
        instruments_df.loc[i, 'EU'] = 'No'
instruments_df=instruments_df[["currency", "instrument_uid",'emdm','rating','class','securit','EU']]   
instruments_df.head()

,currency,instrument_uid,emdm,rating,class,securit,EU
0,GBP,LUID_00003HI5,DM,AA,NA,No,No
1,ZAR,LUID_00003DX5,EM,B,NA,No,No
2,ZAR,LUID_00003DX2,EM,B,NA,No,No
3,ZAR,LUID_00003DWT,EM,B,NA,No,No
4,ZAR,LUID_00003DWS,EM,B,NA,No,No


In [60]:
ins=instruments_api.get_instruments(identifier_type='LusidInstrumentId',scope='BG_PMS_DEMO',request_body=instruments_df['instrument_uid'].to_list()
)

pms_demo_inst=lusid_response_to_data_frame(list(ins.values.values()))[['lusid_instrument_id']]
pms_demo_inst=pd.merge(pms_demo_inst,instruments_df,how='inner',left_on='lusid_instrument_id',right_on='instrument_uid')

default_ins = instruments_df[~instruments_df['instrument_uid'].isin(pms_demo_inst['lusid_instrument_id'])]


In [61]:
requests = []

for row in pms_demo_inst.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property1 = lm.ModelProperty(
        key=f"Instrument/{scope}/EMDM",
        value=lm.PropertyValue(
            label_value=instrument['emdm'])
    )    

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property1])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(scope='BG_PMS_DEMO',
    upsert_instrument_property_request=requests)

requests = []

for row in default_ins.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property1 = lm.ModelProperty(
        key=f"Instrument/{scope}/EMDM",
        value=lm.PropertyValue(
            label_value=instrument['emdm'])
    )    

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property1])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(
    upsert_instrument_property_request=requests)

In [ ]:
requests = []

for row in pms_demo_inst.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property2= lm.ModelProperty(
        key=f"Instrument/{scope}/SecDesc",
        value=lm.PropertyValue(
            label_value=instrument['class'])
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property2])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(scope='BG_PMS_DEMO',
    upsert_instrument_property_request=requests)
        
requests = []

for row in default_ins.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property2= lm.ModelProperty(
        key=f"Instrument/{scope}/SecDesc",
        value=lm.PropertyValue(
            label_value=instrument['class'])
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property2])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(
    upsert_instrument_property_request=requests)

In [62]:
requests = []

for row in pms_demo_inst.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property3= lm.ModelProperty(
        key=f"Instrument/{scope}/Ratings",
        value=lm.PropertyValue(
            label_value=instrument['rating'])
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property3])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(scope='BG_PMS_DEMO',
    upsert_instrument_property_request=requests)
        
requests = []

for row in default_ins.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property3= lm.ModelProperty(
        key=f"Instrument/{scope}/Ratings",
        value=lm.PropertyValue(
            label_value=instrument['rating'])
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property3])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(
    upsert_instrument_property_request=requests)

In [63]:
requests = []

for row in pms_demo_inst.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property4= lm.ModelProperty(
        key=f"Instrument/{scope}/Securitized",
        value=lm.PropertyValue(
            label_value=instrument['securit'])
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property4])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(scope='BG_PMS_DEMO',
    upsert_instrument_property_request=requests)
        
requests = []

for row in default_ins.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property4= lm.ModelProperty(
        key=f"Instrument/{scope}/Securitized",
        value=lm.PropertyValue(
            label_value=instrument['securit'])
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property4])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(
    upsert_instrument_property_request=requests)

In [ ]:
requests = []

for row in pms_demo_inst.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property5= lm.ModelProperty(
        key=f"Instrument/{scope}/MeetsDueDilligencePerEuRegs",
        value=lm.PropertyValue(
            label_value=instrument['EU'])
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property5])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(scope='BG_PMS_DEMO',
    upsert_instrument_property_request=requests)
        
requests = []

for row in default_ins.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property5= lm.ModelProperty(
        key=f"Instrument/{scope}/MeetsDueDilligencePerEuRegs",
        value=lm.PropertyValue(
            label_value=instrument['EU'])
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property5])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(
    upsert_instrument_property_request=requests)

In [64]:
requests = []

for row in pms_demo_inst.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property6= lm.ModelProperty(
        key=f"Instrument/{scope}/Placeholder",
        value=lm.PropertyValue(
            label_value='')
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property6])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(scope='BG_PMS_DEMO',
    upsert_instrument_property_request=requests)
        
requests = []

for row in default_ins.iterrows():
    # Collect our instrument data from the row, note row[0] is the index of the row e.g. 3
    instrument = row[1]
    
    # Create our security_type property for this instrument
    asset_class_property6= lm.ModelProperty(
        key=f"Instrument/{scope}/Placeholder",
        value=lm.PropertyValue(
            label_value='')
    ) 

    # Build our request to set our property
    requests.append(
        lm.UpsertInstrumentPropertyRequest(
            identifier_type='LusidInstrumentId',
            identifier=instrument['instrument_uid'],
            properties=[asset_class_property6])
    )

# Call the LUSID API to add our property across all instruments
response = api_factory.build(lusid.api.InstrumentsApi).upsert_instruments_properties(
    upsert_instrument_property_request=requests)

### 3 Load instruments
create an EM bond, DM bond, HY bond, Index futures and a MBS

We will use these to raise orders later on

In [107]:
bond_definition1 = lm.Bond(
    start_date="2023-11-01",
    maturity_date="2028-06-30",
    dom_ccy="GBP",
    flow_conventions=lm.FlowConventions(
        currency="GBP",
        payment_frequency="6M",
        day_count_convention="Act365",
        roll_convention="None",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=2,
        reset_days=2,
    ),
    principal=1,
    coupon_rate=0.04,
    identifiers={},
    instrument_type="Bond",
)

bond_definition2 = lm.Bond(
    start_date="2023-09-01",
    maturity_date="2047-09-01",
    dom_ccy="KRW",
    flow_conventions=lm.FlowConventions(
        currency="KRW",
        payment_frequency="6M",
        day_count_convention="Act365",
        roll_convention="ModifiedFollowing",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=2,
        reset_days=2,
    ),
    principal=1,
    coupon_rate=0.07,
    identifiers={},
    instrument_type="Bond",
)

bond_definition3 = lm.Bond(
    start_date="2023-11-01",
    maturity_date="2030-04-27",
    dom_ccy="ZAR",
    flow_conventions=lm.FlowConventions(
        currency="ZAR",
        payment_frequency="6M",
        day_count_convention="Act365",
        roll_convention="None",
        payment_calendars=[],
        reset_calendars=[],
        settle_days=2,
        reset_days=2,
    ),
    principal=1,
    coupon_rate=0.18,
    identifiers={},
    instrument_type="Bond",
)

future1 = lm.Future(
    start_date="2024-02-01",
    maturity_date="2024-04-01",
    contract_details=lm.FuturesContractDetails(
        contract_code="FMCH",
        dom_ccy="CNY",
        contract_month="Z",
        contract_size=10000,
        convention='ActualActual',
        country='CN',
        description='MSCI China future Mar 24',
        exchange_code='EUREX',
        exchange_name='Eurex',
        ticker_step=0.01,
        unit_value=10,
        
    ),
    contracts=1,
    ref_spot_price=1,
    identifiers={},
    instrument_type="Future",
)


upsert_instrument = instruments_api.upsert_instruments(
    request_body={
        "ABDNLN 4 1/4 06/30/28": lm.InstrumentDefinition(
            name="ABDNLN 4 1/4 06/30/28",
            identifiers={"ClientInternal": lm.InstrumentIdValue(value="XS1698906259")},
            properties=[lm.ModelProperty(f"Instrument/{scope}/EMDM", value=lm.PropertyValue(label_value="DM"))],
            definition=bond_definition1),
        "CJEXKR 7 1/4 01/09/47": lm.InstrumentDefinition(
            name="CJEXKR 7 1/4 01/09/47",
            identifiers={"ClientInternal": lm.InstrumentIdValue(value="KR6000123861")},
            properties=[lm.ModelProperty(f"Instrument/{scope}/EMDM", value=lm.PropertyValue(label_value="EM"))],
            definition=bond_definition2),
         "RSGVSA 18 3/4 04/27/30": lm.InstrumentDefinition(
            name="RSGVSA 18 3/4 04/27/30",
            identifiers={"ClientInternal": lm.InstrumentIdValue(value="ZAG000016320")},
            properties=[lm.ModelProperty(f"Instrument/{scope}/Ratings", value=lm.PropertyValue(label_value="B"))],
            definition=bond_definition3),
        "MSCI China Future Mar 24": lm.InstrumentDefinition(
            name="MSCI China Future Mar 24",
            identifiers={"ClientInternal": lm.InstrumentIdValue(value="DE000A3CWPD4")},
            properties=[lm.ModelProperty(f"Instrument/{scope}/SecDesc", value=lm.PropertyValue(label_value="FutIndex"))],
            definition=future1),
    }
)
upsert_instruments_response_df = lusid_response_to_data_frame(list(upsert_instrument.values.values()))
display(upsert_instruments_response_df[["name", "lusid_instrument_id"]])

,name,lusid_instrument_id
0,MSCI China Future Mar 24,LUID_00003HI3
1,ABDNLN 4 1/4 06/30/28,LUID_00003H2X
2,CJEXKR 7 1/4 01/09/47,LUID_00003HI1
3,RSGVSA 18 3/4 04/27/30,LUID_00003HI2


In [108]:
def create_fixed_rate_Fannie_Mae_MBS(
   bond_name,
   ISIN,
   currency,
   start_date,
   maturity_date,
   original_notional,
   coupon,
   pay_delay,
   payment_frequency,
   roll_convention,
   stub_type
):
    bond_flow_conventions = lm.FlowConventions(
            currency = currency,
            payment_frequency = payment_frequency,
            roll_convention = roll_convention,
            day_count_convention = 'Thirty360',
            payment_calendars = [],
            reset_calendars = []
    )

    pay_delay_information = lm.ExDividendConfiguration(
        ex_dividend_days = pay_delay,
        apply_thirty360_pay_delay = True
    )

    fixed_schedule = lm.FixedSchedule(
        schedule_type = "FixedSchedule",
        start_date = start_date,
        maturity_date = maturity_date,
        coupon_rate = coupon,
        notional = original_notional,
        payment_currency = currency,
        stub_type = 'None',
        flow_conventions = bond_flow_conventions,
        ex_dividend_configuration = pay_delay_information
    )

    bond = lm.ComplexBond(
        asset_backed = True,
        asset_pool_identifier = ISIN,
        instrument_type = "ComplexBond",
        schedules = [fixed_schedule]
    )

    # define the instrument to be upserted
    bond_definition = lm.InstrumentDefinition(
        name=bond_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(ISIN)},
        definition=bond,
    )

    # upsert the instrument
    upsert_request = {ISIN: bond_definition}
    upsert_response = instruments_api.upsert_instruments(request_body=upsert_request)
    bond_luid = upsert_response.values[ISIN].lusid_instrument_id
    print(f"Bond {ISIN} upserted successfully - LUID is {bond_luid}")

In [109]:
coupon_rate = 0.025
bond_name = 'FED NATIONAL MTGE ASSOC 2014-33'
ISIN = 'US3136AKAD59'
currency = 'USD'
start_date = datetime(2023, 5, 1, 00, tzinfo=pytz.utc)
maturity_date = datetime(2029, 9, 25, 00, tzinfo=pytz.utc)
original_notional = 1000000
pay_delay = 24
payment_frequency = '1M'
roll_convention = 'Day25'
stub_type = 'None'

create_fixed_rate_Fannie_Mae_MBS(
   bond_name,
   ISIN,
   currency,
   start_date,
   maturity_date,
   original_notional,
   coupon_rate,
   pay_delay,
   payment_frequency,
   roll_convention,
   stub_type
)

Bond US3136AKAD59 upserted successfully - LUID is LUID_00003HI4


### 3 create reference lists
Create some list in order to use it in the compliance rules. 

In [155]:
request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='portfolio-list'
    ),
    name="portfolioid list",
    description="EM bond portfolio",
    tags=[],
    reference_list=lm.PortfolioIdList(
        reference_list_type='PortfolioIdList',
        values=[]
    )
)
#lm.ResourceId(scope=scope, code = port_code)
portfolioid_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)

request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='restricted-assets'
    ),
    name="restricted assets list",
    description="restricted assets",
    tags=[],
    reference_list=lm.StringList(
        reference_list_type='StringList',
        values=["Yes"]
    )
)
empty_portfolioid_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)

request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='EM-assets'
    ),
    name="EM assets",
    description="EM assets",
    tags=[],
    reference_list=lm.StringList(
        reference_list_type='StringList',
        values=["DM"]
    )
)
empty_portfolioid_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)

request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='high-risk-bond-holdings'
    ),
    name="high risk bond holdings",
    description="list of ratings classified as high risk",
    tags=[],
    reference_list=lm.StringList(
        reference_list_type='StringList',
        values=["BB",'B','CCC','CC','C','D']
    )
)
empty_portfolioid_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)

request = ReferenceListRequest(
    id=lm.ResourceId(
        scope=scope,
        code='empty-list'
    ),
    name="empty string list",
    description="All asset classes",
    tags=[],
    reference_list=lm.StringList(
        reference_list_type='StringList',
        values=[]
    )
)
empty_portfolioid_list_response = referencelist_api.upsert_reference_list_with_http_info(reference_list_request=request)


### 4 Create compliance rules
Create some list in order to use it in the compliance rules. 

In [156]:
# do not invest in chinese stock index futures
upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='rule-1'),
    name="restricted-asset-check",
    description='no investment into chinese stock index futures',
    template_id=lm.ResourceId(scope='system', code='PropertyValueNotInList'),
    variation='standard-not-in-list',
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "PropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/BG_PMS_DEMO/CNFtIndex", compliance_parameter_type='PropertyKeyComplianceParameter'),
        "ExclusivePropertyList": lm.StringListComplianceParameter(value=lm.ResourceId(scope=scope, code="restricted-assets"),compliance_parameter_type='StringListComplianceParameter')
    },
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)

In [ ]:
#not buy securitization bonds issued after 1st jan 2019 unless satisfy 
#EU due diligence regulation
upsert_compliance_rule_request = UpsertComplianceRuleRequest(
    id=lm.ResourceId(scope=scope, code='rule-2'),
    name="restricted-asset-check",
    description='no investment into securitised bonds unless approved by EU regulation',
    template_id=lm.ResourceId(scope='system', code='PropertyValueNotInList'),
    variation='standard-not-in-list',
    portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
    active=True,
    parameters={
        "PropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/BG_PMS_DEMO/ApprovedSecuritisation", compliance_parameter_type='PropertyKeyComplianceParameter'),
        "ExclusivePropertyList": lm.StringListComplianceParameter(value=lm.ResourceId(scope=scope, code="restricted-assets"),compliance_parameter_type='StringListComplianceParameter')
    },
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)

In [157]:
#EM bond minimum holdings
pvInReportCcy = lm.AddressKeyComplianceParameter("passed_validation", compliance_parameter_type='AddressKeyComplianceParameter')
pvInReportCcy._value = "Valuation/PvInReportCcy"
rule_code = 'rule-3'
rule_name = 'Min EM bond exposure'
rule_description = 'Set bounds on EM Bond Exposure'
upper_bound = 20
lower_bound = -1
upper_warning = 20
lower_warning = -1
grouping_property = "Instrument/BG_PMS_DEMO/EMDM"
metric = pvInReportCcy

upsert_compliance_rule_request = UpsertComplianceRuleRequest(   
id=lm.ResourceId(scope=scope, code=rule_code),
name=rule_name,
description=rule_description,
template_id=lm.ResourceId(scope='system', code='PercentCheck'),
variation='standard',
portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
active=True,
parameters={
    "Metric": metric,
    "UpperBound": lm.DecimalComplianceParameter(value=str(upper_bound),compliance_parameter_type='DecimalComplianceParameter'),
    "LowerBound": lm.DecimalComplianceParameter(value=str(lower_bound),compliance_parameter_type='DecimalComplianceParameter'),
    "UpperWarning": lm.DecimalComplianceParameter(value=str(upper_warning),compliance_parameter_type='DecimalComplianceParameter'),
    "LowerWarning": lm.DecimalComplianceParameter(value=str(lower_warning),compliance_parameter_type='DecimalComplianceParameter'),
    "FirstFilterPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/{scope}/EMDM",compliance_parameter_type='PropertyKeyComplianceParameter'),
    "FirstFilterPermittedValuesList": lm.StringListComplianceParameter(value=lm.ResourceId(scope=scope, code="EM-assets"),compliance_parameter_type='StringListComplianceParameter'),
    "SecondFilterPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/{scope}/Placeholder",compliance_parameter_type='PropertyKeyComplianceParameter'),
    "SecondFilterPermittedValuesList": lm.StringListComplianceParameter(value=lm.ResourceId(scope=scope, code="empty-list"),compliance_parameter_type='StringListComplianceParameter'),
    "GroupingPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/{scope}/EMDM",compliance_parameter_type='PropertyKeyComplianceParameter'),
    "Excludes": lm.PortfolioIdListComplianceParameter(value=lm.ResourceId(scope=scope,code="portfolio-list"),compliance_parameter_type='PortfolioIdListComplianceParameter')
},
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)


In [159]:
#junk bond exposure limit
rule_code = 'rule-4'
rule_name = 'Max junk bond exposure'
rule_description = 'Set bounds on bonds with ratings lower than BBB- Exposure'
upper_bound = 10
lower_bound = -1
upper_warning = 10
lower_warning = -1
grouping_property = "Instrument/BG_PMS_DEMO/HYbond"
upsert_compliance_rule_request = UpsertComplianceRuleRequest(   
id=lm.ResourceId(scope=scope, code=rule_code),
name=rule_name,
description=rule_description,
template_id=lm.ResourceId(scope='system', code='PercentCheck'),
variation='standard',
portfolio_group_id=lm.ResourceId(scope=scope, code=portfolio_group_code),
active=True,
parameters={
    "Metric": metric,
    "UpperBound": lm.DecimalComplianceParameter(value=str(upper_bound),compliance_parameter_type='DecimalComplianceParameter'),
    "LowerBound": lm.DecimalComplianceParameter(value=str(lower_bound),compliance_parameter_type='DecimalComplianceParameter'),
    "UpperWarning": lm.DecimalComplianceParameter(value=str(upper_warning),compliance_parameter_type='DecimalComplianceParameter'),
    "LowerWarning": lm.DecimalComplianceParameter(value=str(lower_warning),compliance_parameter_type='DecimalComplianceParameter'),
    "FirstFilterPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/{scope}/HYbond",compliance_parameter_type='PropertyKeyComplianceParameter'),
    "FirstFilterPermittedValuesList": lm.StringListComplianceParameter(value=lm.ResourceId(scope=scope, code="restricted-assets"),compliance_parameter_type='StringListComplianceParameter'),
    "SecondFilterPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/{scope}/Placeholder",compliance_parameter_type='PropertyKeyComplianceParameter'),
    "SecondFilterPermittedValuesList": lm.StringListComplianceParameter(value=lm.ResourceId(scope=scope, code="empty-list"),compliance_parameter_type='StringListComplianceParameter'),
    "GroupingPropertyKey": lm.PropertyKeyComplianceParameter(value=f"Instrument/{scope}/HYbond",compliance_parameter_type='PropertyKeyComplianceParameter'),
    "Excludes": lm.PortfolioIdListComplianceParameter(value=lm.ResourceId(scope=scope,code="portfolio-list"),compliance_parameter_type='PortfolioIdListComplianceParameter')
},
    properties={}
)

compliance_api.upsert_compliance_rule(upsert_compliance_rule_request=upsert_compliance_rule_request)


### 5 Run and analyze the rules
Create some list in order to use it in the compliance rules. 

In [167]:
run_response = compliance_api.run_compliance(run_scope=scope,rule_scope=scope,is_pre_trade=True,recipe_id_scope='BG_PMS_DEMO',recipe_id_code='lookthrough')
run_code = run_response.run_id.code

print(f"Compliance run {run_response.run_id.scope}/{run_response.run_id.code} completed.")
print(f"Instigated: {run_response.instigated_at}, completed: {run_response.completed_at}")

Compliance run BG_PMS_DEMO/c38542e4-1d0c-4eb9-8c8e-a37498479ee3 completed.
Instigated: 2024-02-16 16:26:56.521420+00:00, completed: 2024-02-16 16:27:00.038630+00:00


In [168]:
def rule_level_dataframe(run_summary):
    # Use the first result as a way of generating overall headers
    h = ['', '', '', '', '']
    c = ['Rule', 'Rule Description', 'Status', 'Affected Orders', 'Affected Portfolios']

    df = pd.DataFrame([c], columns=h)

    new_labels = pd.MultiIndex.from_arrays([df.columns, df.iloc[0]], names=['', ''])
    df = df.set_axis(new_labels, axis=1).iloc[1:]

    # Now build a row per result
    for d in run_summary.details:
        r = [f"{d.rule_id.scope}/{d.rule_id.code}", d.rule_description, d.status, len(d.affected_orders), len(d.affected_portfolios_details)]

        df.loc[len(df)] = r

    return df

In [169]:
run_summary = compliance_api.get_decorated_compliance_run_summary(scope=scope, code=run_code)
df = rule_level_dataframe(run_summary)
display(df)

\
                 Rule                                   Rule Description   
0  BG_PMS_DEMO/rule-2  no investment into securitised bonds unless ap...   
1  BG_PMS_DEMO/rule-3                     Set bounds on EM Bond Exposure   
2  BG_PMS_DEMO/rule-1     no investment into chinese stock index futures   
3  BG_PMS_DEMO/rule-4  Set bounds on bonds with ratings lower than BB...   

                                               
   Status Affected Orders Affected Portfolios  
0  Passed               0                   1  
1  Failed               0                   1  
2  Passed               0                   1  
3  Failed               0                   1

In [34]:
rule_result1 = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code, rule_scope=scope, rule_code='rule-3')
rule_result1
filtered_breakdowns1 = [breakdown for breakdown in rule_result1.rule_result.rule_breakdown if getattr(breakdown, 'group_status', '') == 'Passed']
rule_result1_df=lusid_response_to_data_frame(filtered_breakdowns1)
rule_result1_df
display(rule_result1_df[['group_status','results_used.Valuation/PvInReportCcy','results_used.Portfolios.Valuation/PvInReportCcy',
                       'lineage.1.label','lineage.2.sub_label','lineage.5.sub_label','lineage.5.information']])

,group_status,results_used.Valuation/PvInReportCcy,results_used.Portfolios.Valuation/PvInReportCcy,lineage.1.label,lineage.2.sub_label,lineage.5.sub_label,lineage.5.information
0,Passed,-3.741350e+07,-6.720553e+10,WithoutExcludedPortfolios,BG_PMS_DEMO/89518,DM,Instrument/BG_PMS_DEMO/EMDM


### 6 Enter some orders to show its effect on the compliance
Create some orders and run pretrade against that

In [68]:
orders_df = pd.read_csv('data/live_orders.csv')
orders_df

,instrument_name,LusidInstrumentId,quantity,price,currency,order_id,side,type,state,limit_price,limit_currency,stop_price,stop_currency
0,ABDNLN 4 1/4 06/30/28,LUID_00003H2X,10000000000,100.0,GBP,ORD005,Buy,Limit,New,100.0,GBP,NaN,NaN
1,RSGVSA 18 3/4 04/27/30,LUID_00003HI2,4000,NaN,ZAR,ORD008,Buy,Market,New,NaN,NaN,NaN,NaN


In [69]:
order_requests = defaultdict(list)
order_sets = defaultdict(list)
responses = []

for index, order in orders_df.iterrows():
    
    request = lm.OrderRequest(
            id=lm.ResourceId(
                scope=scope,
                code=order['order_id']
            ),
            quantity=order['quantity'],
            side=order['side'],
            instrument_identifiers={
                'Instrument/default/LusidInstrumentId': order['LusidInstrumentId']
            },
            properties={},
            portfolio_id=lm.ResourceId(
                scope=scope,
                code=portfolio_code
            ),
            state=order['state'],
            type=order['type'],
            price=lm.CurrencyAndAmount(
                        amount=0 if pd.isna(order['price']) else order['price'],
                        currency=order['currency']
            ),

            limit_price=lm.CurrencyAndAmount(
                        amount=order['limit_price'],
                        currency=order['limit_currency']
            )            
            if not pd.isna(order['limit_price']) and not pd.isna(order['limit_currency']) else None,
            
            stop_price=lm.CurrencyAndAmount(
                        amount=order['stop_price'],
                        currency=order['stop_currency']
            )
            if not pd.isna(order['stop_price']) and not pd.isna(order['stop_currency']) else None
    )
    
    request=lm.OrderSetRequest(
        order_requests=[request]           
    )

    response = api_factory.build(lusid.api.OrdersApi).upsert_orders(
        order_set_request=request
    )
    
    responses.append(response.values[0])

attributes=[(o.id.code,o.instrument_identifiers['Instrument/default/LusidInstrumentId'],
             o.lusid_instrument_id,o.side,o.type,o.state,o.quantity,o.price.amount,o.price.currency,
             o.limit_price.amount if o.limit_price is not None else "N/A",
             o.limit_price.currency if o.limit_price is not None else "N/A") for o in responses]

pd.DataFrame(attributes, columns=['order_id','client_internal','lusid_instrument_id','side','type',
                                  'state','quantity','price','currency','lim px',
                                  'lim ccy']).style.format({"quantity":"{:20,.0f}","price": "{:20,.2f}"})

,order_id,client_internal,lusid_instrument_id,side,type,state,quantity,price,currency,lim px,lim ccy
0,ORD005,LUID_00003H2X,LUID_00003H2X,Buy,Limit,New,"10,000,000,000",100.00,GBP,100.000000,GBP
1,ORD008,LUID_00003HI2,LUID_00003HI2,Buy,Market,New,"4,000",0.00,ZAR,N/A,N/A


In [70]:
order_response = compliance_api.run_compliance(run_scope=scope,rule_scope=scope,is_pre_trade=True,recipe_id_scope='BG_PMS_DEMO',recipe_id_code='MarketValuation')
run_code_order = order_response.run_id.code

print(f"Compliance run {run_response.run_id.scope}/{run_response.run_id.code} completed.")
print(f"Instigated: {run_response.instigated_at}, completed: {run_response.completed_at}")

Compliance run BG_PMS_DEMO/ab0ea218-18f4-4dd6-891f-e49161e83f3d completed.
Instigated: 2024-02-15 10:37:28.368614+00:00, completed: 2024-02-15 10:37:30.424779+00:00


In [71]:
run_summary_order = compliance_api.get_decorated_compliance_run_summary(scope=scope, code=run_code_order)
df_order = rule_level_dataframe(run_summary_order)
display(df_order)

\
                 Rule                                   Rule Description   
0  BG_PMS_DEMO/rule-3                     Set bounds on EM Bond Exposure   
1  BG_PMS_DEMO/rule-4  Set bounds on bonds with ratings lower than BB...   
2  BG_PMS_DEMO/rule-1     no investment into chinese stock index futures   
3  BG_PMS_DEMO/rule-2  no investment into securitised bonds unless ap...   

                                               
   Status Affected Orders Affected Portfolios  
0  Failed               2                   1  
1  Passed               2                   1  
2  Failed               0                   1  
3  Failed               1                   1

In [67]:
rule_result3 = compliance_api.get_compliance_rule_result(run_scope=scope, run_code=run_code_order, rule_scope=scope, rule_code='rule-3')
filtered_breakdowns_orders = [breakdown for breakdown in rule_result3.rule_result.rule_breakdown if getattr(breakdown, 'group_status', '') == 'Failed']


order_result_df=lusid_response_to_data_frame(filtered_breakdowns_orders)
display(order_result_df[['group_status','results_used.Valuation/PvInReportCcy','results_used.Portfolios.Valuation/PvInReportCcy',
                       'lineage.1.label','lineage.2.sub_label','lineage.5.sub_label','lineage.5.information']])

,group_status,results_used.Valuation/PvInReportCcy,results_used.Portfolios.Valuation/PvInReportCcy,lineage.1.label,lineage.2.sub_label,lineage.5.sub_label,lineage.5.information
0,Failed,8.534515e+14,-6.720553e+10,WithoutExcludedPortfolios,BG_PMS_DEMO/89518,DM,Instrument/BG_PMS_DEMO/EMDM
